# Setting up the analysis

## Getting the data on the sample firms

In [1]:
import glob
import os

import pandas as pd
from sec_edgar_downloader import Downloader
from tqdm import tqdm 
import shutil

In [2]:
# places to put files - best practice chapter 2!

os.makedirs("inputs", exist_ok=True)
os.makedirs("10k_files", exist_ok=True)

## Step 1: Get the S&P 500 firms

In a more rigorous project, we would use all publicly traded firms with available 10-Ks and returns. This is more firms than I want students to manage on the midterm.

Using a sample of S&P500 firms is a sensible compromise. Two major points come up, the first of which we discussed in class a lot.

1. The obvious limitation is "what if the relationships we find are different for smaller firms outside the S&P500"? This is a good concern, and worthy of discussion in your results. Do you have an **economic argument** for why your particular relationships would be more or less relevant for small firms (than for the larger firms in the S&P500)? Depending on your answer, that means a relationship you find might be too high or too low. Maybe the sign of the relationship flips. 
1. Making sure we get the 2022 list of S&P500 firms. We do this by adjusting the URL to use the wiki page as of year end 2022. **As a result of this, we have 2022 tickers, which can cause problems:** If firms go private, fail, are acquired, or change their ticker between 2022 and now, then (A) our SEC scraper might not find the filings for that firm and (B) we might struggle to link the filings to other datasets like CRSP and Compustat.**To avoid this, we will tell the scraper to search for firms using their CIK**
 


In [3]:
# path and place to put it
sp500_file = 'inputs/sp500_2022.csv'

# get it if we haven't 
if not os.path.exists(sp500_file):
    # 2022 dec version of page
    url = 'https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=1130173030'
    pd.read_html(url)[0].to_csv(sp500_file,index=False)

# load and look at it
sp500 = pd.read_csv(sp500_file)    

In [4]:
sp500

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


## Step 2: Download their last 10-K 

For me, this took 6 seconds per download, or roughly 30 minutes overall. Previously, it took 45 minutes. 

I was able to download a 10-K for 498 of the 503 firms.

This code here does not attempt to fix or explore why five 10-Ks are missing. Do you know why?



In [5]:
dl = Downloader("Lehigh", 
                "deb219@lehigh.edu",
                "10k_files")

In [6]:
if not os.path.exists('10k_files/10k_files.zip'):
    
    for cik in tqdm(sp500['CIK'][:10]): # start with a small subset while we figure things out, remove later
         
        firm_folder = f'10k_files/sec-edgar-filings/{str(cik).zfill(10)}/'  # str(cik).zfill(10)   means that CIK 1234 becomes 0000001234

        # if I haven't downloaded any HTML for this firm (len=0 files on this pattern), do so
        # you could make this more precise and only look for filings during 2022 (if you were downloading across many years)
        
        if len(glob.glob(firm_folder + '/10-K/*/*.html')) == 0:
            
            dl.get("10-K", cik, 
                   limit=1,                  # get the latest filing within window
                   after="2022-01-01",       # does this download filings ON 1/1 or nah? (check)
                   before="2022-12-31",      # does this download filings ON 12/31 or nah? (check)
                   download_details =True    # download the html 
            ) 
    
        # delete the txt files as we go!!!
        # files are of the form: folder/10-K/*/*.txt
        for txt_f in glob.glob(firm_folder + '/10-K/*/*.txt'):
            os.remove(txt_f)    
    
        # pause if there is a problem and the SEC is mad at my spider
        # unneeded! sec-edgar-dl does it for us 


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.20s/it]


Stop! Check. ABCD. Do we have enough files? Why are any missing files missing? Is it ok to move on, or are fixes needed?

In [7]:
files = glob.glob('10k_files/sec-edgar-filings/*/10-K/*/*.html')

f'We have {len(files)} HTML files for {len(sp500["CIK"])} firms'


'We have 10 HTML files for 503 firms'

## Step 3: Reduce hard drive usage

Don't run this until you are done with downloads. What is below is a "one shot" code. Use it once only.

There is an explicit variable controlling it. If you set `done_with_downloads=True` it will create the zip file and delete all the raw files. If you leave as `False`

In [8]:
# set to True to run the code below. make sure you are done with downloads first!
# see if your folder has ~500ish html files, and take the screenshot from instructions
done_with_downloads = True 

if os.path.exists('10k_files/sec-edgar-filings') and \
    not os.path.exists('10k_files/10k_files.zip') and \
    done_with_downloads:
    
    # zip the folder (2GB --> 150mb)
    shutil.make_archive('10k_files', 'zip', '10k_files')
    
    # delete the folder with all the files
    # I've commented this out for now so that we can keep opening up the 10-Ks up easily 
    #shutil.rmtree('10k_files/sec-edgar-filings')
    
    # put the zip file in the `10k_files` folder
    shutil.move('10k_files.zip', '10k_files/')